### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Psoriasis/GSE162926'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# 1. Gene Expression Data Availability
is_gene_available = False  # Since this dataset focuses on miRNA, it's not suitable for gene expression analysis.

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Check the sample characteristics dictionary
sample_characteristics_dict = {
    0: ['subject status: atopic dermatitis adult patient', 'subject status: healthy control adult', 'subject status: psoriasis adult patient'],
    1: ['tissue: serum']
}

# Identify the key for 'Psoriasis'
for key, values in sample_characteristics_dict.items():
    if any('psoriasis' in value for value in values):
        trait_row = key
        break

# Since 'age' and 'gender' are not explicitly defined, and there is no sufficient inferable data, they are set to None.
age_row = None
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    trait_mapping = {
        'subject status: psoriasis adult patient': 1,
        'subject status: healthy control adult': 0,
    }
    value = value.split(': ')[1]
    return trait_mapping.get(value, None)

def convert_age(value):
    return None  # no conversion logic since age data is not available

def convert_gender(value):
    return None  # no conversion logic since gender data is not available

# 3. Save Metadata
from utils.preprocess import save_cohort_info
save_cohort_info('GSE162926', './preprocessed/Psoriasis/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction (Only if clinical data is available)
if trait_row is not None:
    from utils.preprocess import geo_select_clinical_features
    import pandas as pd
    # Load the actual clinical data from appropriate source
    clinical_data = pd.read_csv('path_to_clinical_data.csv')  # Make sure to replace 'path_to_clinical_data.csv' with the actual path
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Psoriasis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Psoriasis/trait_data/GSE162926.csv'
    selected_clinical_data.to_csv(csv_path)
    from utils.preprocess import preview_df
    print(preview_df(selected_clinical_data))
